In [1]:
from flask import Flask, render_template, url_for, request, session, redirect
from tensorflow.keras.applications.imagenet_utils import preprocess_input, decode_predictions
from werkzeug.utils import secure_filename
from tensorflow.keras.models import load_model
import os
from flask_pymongo import PyMongo
from tensorflow.keras.preprocessing import image
import smtplib

import bcrypt
import pickle
import numpy as np
app = Flask(__name__)

mongo1 = PyMongo(app, uri = 'mongodb://127.0.0.1:27017/Patient')
mongo2 = PyMongo(app, uri = 'mongodb://127.0.0.1:27017/doctors')

In [2]:
s = smtplib.SMTP('smtp.gmail.com', 587)
s.starttls()
my_id = "iamepicwin80@gmail.com"
s.login(my_id, "sucheet2000")

(235, b'2.7.0 Accepted')

In [3]:
pickle_in = open("classifier_heart.pkl","rb")
classifier_heart_disease=pickle.load(pickle_in)
print("done")
pickle_in = open("breast_cancer_model.pkl","rb")
breast_cancer_model=pickle.load(pickle_in)
print("done")
MODEL_PATH ='cnn_model.h5'
dengue_model = load_model(MODEL_PATH)
print("done")
pickle_in = open("classifier_diabetes.pkl","rb")
classifier_diabetes_disease=pickle.load(pickle_in)

done
done
done


In [4]:
@app.route('/',methods = ['POST','GET'])
def index():
    if 'email' in session:
        return render_template('iwp_features.html')    
    return render_template('index.html')

In [5]:
@app.route('/to_decider',methods = ['POST','GET'])
def to_decider():
    if request.method=="GET":
        return render_template('patient_or_doctor.html')    

In [6]:
@app.route('/decider',methods = ['POST','GET'])
def decider():
    if request.method=="POST":
        
        roll = request.form['roll']
        if roll=="doctor":
            return render_template('register_doctor.html')
        else:
            return render_template('register_patient.html')

In [7]:
@app.route('/selected_disease',methods = ['POST','GET'])
def selected_disease():
    if request.method=="POST":
        
        disease = request.form['Disease']
        if disease == "heart_disease":
            return render_template('iwp_features.html')
        if disease == "dengue":
            return render_template('dengue.html')
        if disease == "diabetes":
            return render_template('diabetes.html')
        if disease == "breast_cancer":
            return render_template('breast_cancer.html')

In [8]:
@app.route('/predict_heart_disease',methods=["POST"])
def predict_heart_disease():
    users = mongo1.db.collection
    login_user = users.find_one({'email' : session['email']})
    if request.method == "POST":
        Age = request.form["Age"]
        Cholestrol = request.form["Cholestrol"]
        oldpeak = request.form["oldpeak"]
        Max_HR = request.form["Max_HR"]
        x = np.asarray([[Age,Cholestrol,oldpeak,Max_HR]], dtype='float64')
        prediction = str(classifier_heart_disease.predict(x))
        if prediction=='[0]':
            prediction = "No heart disease"
            msg = "Subject: %s\n\n%s"%("Heart report", "you are healthy" )
        else:
            prediction = "HEART DISEASE PREDICTED"
            msg = "Subject: %s\n\n%s"%("Heart report", "Please see a Doctor" )
        s.sendmail(my_id, session['email'], msg)
    
        users.update_one({"email":session['email']},{"$set":{"status":prediction}})
        return render_template("iwp_features.html",msg = "Result: "+prediction)
    return None

In [9]:
@app.route('/predict_diabetes',methods=["POST"])
def predict_diabetes():
    users = mongo1.db.collection
    login_user = users.find_one({'email' : session['email']})
    if request.method == "POST":
        Glucose = request.form["Glucose"]
        BMI = request.form["BMI"]
        Age = request.form["Age"]
        x = np.asarray([[Glucose, BMI, Age]], dtype='float64')
        prediction = str(classifier_diabetes_disease.predict(x))
        if prediction=='[0]':
            prediction = "You are Safe"
            msg = "Subject: %s\n\n%s"%("Diabetes report", "you are healthy" )
        else:
            prediction = "Diabetes detected"
            msg = "Subject: %s\n\n%s"%("Diabetes report", "Please see a doctor, you are suffering from diabetes" )
        s.sendmail(my_id, session['email'], msg)
        users.update_one({"email":session['email']},{"$set":{"status":prediction}})
        return render_template("diabetes.html",msg = "Result: "+prediction)
    return None

In [10]:
@app.route('/predict_breast_cancer',methods=["POST"])
def predict_breast_cancer():
    print("inside breast cancer")
    users = mongo1.db.collection
    login_user = users.find_one({'email' : session['email']})
    if request.method == "POST":
        radius_mean = request.form["radius_mean"]
        perimeter_mean = request.form["perimeter_mean"]
        perimeter_worst = request.form["perimeter_worst"]
        x = np.asarray([[radius_mean,perimeter_mean,perimeter_worst]], dtype='float64')
        prediction = str(breast_cancer_model.predict(x))
        if prediction=='[0]':
            prediction = "Benign"
            msg = "Subject: %s\n\n%s"%("Breast cancer report", "you are healthy" )
        else:
            prediction = "Melignant"
            
            msg = "Subject: %s\n\n%s"%("Breast cancer report", "See a doctor, You're suffering from breast cancer" )
            
        s.sendmail(my_id, session['email'], msg)
        users.update_one({"email":session['email']},{"$set":{"status":prediction}})
        return render_template("breast_cancer.html",msg = "Result: "+prediction)
    return None

In [11]:
@app.route('/predict_dengue', methods=['GET', 'POST'])
def predict_dengue():
    msg = ""
    if request.method == 'POST':
        f = request.files['file']
        basepath = os.path.dirname(f.filename)
        file_path = os.path.join(basepath, 'uploads', secure_filename(f.filename))
        f.save(file_path)
        preds,msg = d_predict(file_path,dengue_model)
        session['status']+=preds
        
        s.sendmail(my_id, session['email'], msg)
        return render_template("dengue.html",msg = "Result: "+preds)
    return None

In [12]:
def d_predict(file_path,model):
    img = image.load_img(file_path,target_size=(224, 224))
    x = image.img_to_array(img)
    x = x/255
    x = np.expand_dims(x, axis=0)
    preds = model.predict(x)
    preds=np.argmax(preds, axis=1)
    if preds==0:
        preds = "INFECTED"
        msg = "Subject: %s\n\n%s"%("Dengue report", "You're infected with dengue. Please see a doctor" )
    else:
        preds = "uninfected"
        msg = "Subject: %s\n\n%s"%("Dengue report", "you are healthy" )
    return preds,msg

In [13]:
@app.route('/login', methods=['POST','GET'])
def login():
    msg=""
    
    users_patient = mongo1.db.collection
    login_patients = users_patient.find_one({'email' : request.form['email']})
    
    if login_patients:
        if bcrypt.hashpw(request.form['password'].encode('utf-8'), login_patients['password']) == login_patients['password']:
            session['email'] = request.form['email']
            session['Name'] = login_patients['Patient Name']
            session['status'] = login_patients['status']
            print("logged in")
            return render_template('select_disease.html')
        
    users_doctor = mongo2.db.collection
    login_doctors = users_doctor.find_one({'email' : request.form['email']})
    if login_doctors:
        if bcrypt.hashpw(request.form['password'].encode('utf-8'), login_doctors['password']) == login_doctors['password']:
            session['email'] = request.form['email']
            display = []
            for record in users_patient.find({},{"_id":0,"Patient Name":1,"Address":1,"email":1 }):
                display.append(record)
            #display = users_patient.find({},{"_id":0,"Patient Name":1,"Address":1,"email":1 })
            return render_template("doc_feat.html",display =  display)
            
    msg = 'Invalid username/password combination'
    return render_template('index.html',msg = msg)    

In [14]:
@app.route('/register_patient', methods=['POST', 'GET'])
def register_patient():
    msg = ""
    if request.method == 'POST':
        patients = mongo1.db.collection
        existing_patients = patients.find_one({'email' : request.form['email']})

        if existing_patients is None:
            hashpass = bcrypt.hashpw(request.form['password'].encode('utf-8'), bcrypt.gensalt())
            patients.insert_one({'Patient Name' : request.form['Name'],'email':request.form['email'], 'password' : hashpass,'Address':request.form['Address'],'status':'unknown'})
            session['email'] = request.form['email']
            session['status'] = 'unknown'
            msg = "you have successfully registered"
            #return redirect(url_for('index'))  #sending to index function
            return render_template('index.html',msg=msg)
        return 'That email already exists!'

    return render_template('register_patient.html',msg = msg)

In [15]:
@app.route('/register_doctor', methods=['POST', 'GET'])
def register_doctor():
    msg = ""
    if request.method == 'POST':
        doctors = mongo2.db.collection
        existing_doctors = doctors.find_one({'email' : request.form['email']})

        if existing_doctors is None:
            hashpass = bcrypt.hashpw(request.form['password'].encode('utf-8'), bcrypt.gensalt())
            doctors.insert_one({'Patient Name' : request.form['Name'],'email':request.form['email'], 'password' : hashpass,'Address':request.form['Address']})
            session['email'] = request.form['email']
            session['status'] = 'unknown'
            msg = "you have successfully registered"
            #return redirect(url_for('index'))  #sending to index function
            return render_template('index.html',msg=msg)
        return 'That email already exists!'

    return render_template('register.html',msg = msg)

In [16]:
@app.route('/logout') 
def logout(): 
    msg=""
    session.pop('email', None) 
    session.pop('Name', None) 
    return render_template('index.html',msg = "Successfully logged out")

In [17]:
@app.route('/trick',methods=["POST"])
def trick():
    
    prediction = "YOU ARE HEALTHY :)"
    return render_template("diabetes.html", msg = "Result: "+prediction)

    '''users = mongo1.db.collection
    login_user = users.find_one({'email' : session['email']})
    if request.method == "POST":
        Glucose = request.form["Glucose"]
        BMI = request.form["BMI"]
        Age = request.form["Age"]
        x = np.asarray([[Glucose, BMI, Age]], dtype='float64')
        prediction = str(classifier_diabetes_disease.predict(x))
        if prediction=='[0]':
            prediction = "You are Safe"
            msg = "Subject: %s\n\n%s"%("Diabetes report", "you are healthy" )
        else:
            prediction = "Diabetes detected"
            msg = "Subject: %s\n\n%s"%("Diabetes report", "Please see a doctor, you are suffering from diabetes" )
        s.sendmail(my_id, session['email'], msg)
        users.update_one({"email":session['email']},{"$set":{"status":prediction}})
        return render_template("diabetes.html",msg = "Result: "+prediction)
    return None'''

In [ ]:
if __name__ == '__main__':
    app.secret_key = 'mysecret'
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [10/Dec/2021 02:17:09] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2021 02:17:09] "GET /styles.css HTTP/1.1" 404 -
127.0.0.1 - - [10/Dec/2021 02:17:09] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [10/Dec/2021 02:17:15] "GET /logout HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2021 02:17:15] "GET /styles.css HTTP/1.1" 404 -
127.0.0.1 - - [10/Dec/2021 02:17:25] "POST /login HTTP/1.1" 200 -


logged in


127.0.0.1 - - [10/Dec/2021 02:17:25] "GET /styles.css HTTP/1.1" 404 -
127.0.0.1 - - [10/Dec/2021 02:17:46] "POST /selected_disease HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2021 02:17:46] "GET /styles.css HTTP/1.1" 404 -


inside breast cancer


...

127.0.0.1 - - [10/Dec/2021 02:20:53] "POST /selected_disease HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2021 02:20:53] "GET /styles.css HTTP/1.1" 404 -
127.0.0.1 - - [10/Dec/2021 02:21:28] "POST /predict_dengue HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2021 02:21:28] "GET /styles.css HTTP/1.1" 404 -
127.0.0.1 - - [10/Dec/2021 02:22:10] "POST /predict_dengue HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2021 02:22:10] "GET /styles.css HTTP/1.1" 404 -
127.0.0.1 - - [10/Dec/2021 02:22:34] "GET /logout HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2021 02:22:34] "GET /styles.css HTTP/1.1" 404 -
127.0.0.1 - - [10/Dec/2021 02:22:42] "POST /login HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2021 02:22:42] "GET /styles.css HTTP/1.1" 404 -


logged in


127.0.0.1 - - [10/Dec/2021 02:22:45] "POST /selected_disease HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2021 02:22:46] "GET /styles.css HTTP/1.1" 404 -
127.0.0.1 - - [10/Dec/2021 02:22:58] "POST /predict_heart_disease HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2021 02:22:59] "GET /styles.css HTTP/1.1" 404 -
127.0.0.1 - - [10/Dec/2021 02:23:42] "POST /predict_heart_disease HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2021 02:23:43] "GET /styles.css HTTP/1.1" 404 -
127.0.0.1 - - [10/Dec/2021 02:24:08] "GET /logout HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2021 02:24:08] "GET /styles.css HTTP/1.1" 404 -
127.0.0.1 - - [10/Dec/2021 02:24:15] "POST /login HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2021 02:24:16] "GET /styles.css HTTP/1.1" 404 -


logged in


127.0.0.1 - - [10/Dec/2021 02:24:18] "POST /selected_disease HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2021 02:24:18] "GET /styles.css HTTP/1.1" 404 -
127.0.0.1 - - [10/Dec/2021 02:24:24] "POST /trick HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2021 02:24:25] "GET /styles.css HTTP/1.1" 404 -
127.0.0.1 - - [10/Dec/2021 02:25:00] "POST /trick HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2021 02:25:00] "GET /styles.css HTTP/1.1" 404 -
127.0.0.1 - - [10/Dec/2021 02:25:11] "POST /trick HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2021 02:25:11] "GET /styles.css HTTP/1.1" 404 -
127.0.0.1 - - [10/Dec/2021 02:25:19] "POST /trick HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2021 02:25:19] "GET /styles.css HTTP/1.1" 404 -
127.0.0.1 - - [10/Dec/2021 02:25:35] "POST /trick HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2021 02:25:35] "GET /styles.css HTTP/1.1" 404 -
127.0.0.1 - - [10/Dec/2021 02:25:48] "POST /trick HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2021 02:25:48] "GET /styles.css HTTP/1.1" 404 -
